# EGFR Bioactivity EDA (Teaching-Style)
This notebook explores your curated EGFR dataset (`EGFR_03_bioactivity_data_curated.csv`). It’s written for non-specialists with step-by-step explanations.

In [ ]:
!pip install rdkit-pypi umap-learn scikit-learn matplotlib pandas numpy --quiet

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from umap import UMAP
from sklearn.feature_selection import f_classif
from sklearn.ensemble import IsolationForest
from rdkit import Chem
from rdkit.Chem import Descriptors
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)

## Load dataset and normalize column names

In [ ]:
csv_path = 'EGFR_03_bioactivity_data_curated.csv'
assert os.path.exists(csv_path), f"Couldn't find {csv_path}. Place it in the working directory."
df = pd.read_csv(csv_path)
if 'molecule_chemble_id' in df.columns and 'molecule_chembl_id' not in df.columns:
    df = df.rename(columns={'molecule_chemble_id': 'molecule_chembl_id'})
print('Shape:', df.shape)
df.head()

## Create pIC50 and quick visuals

In [ ]:
df = df.copy()
df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')
df = df[df['standard_value'] > 0]
df['pIC50'] = 9 - np.log10(df['standard_value'])
df[['standard_value','pIC50']].describe()

In [ ]:
plt.figure(); plt.hist(np.log10(df['standard_value']), bins=50); plt.title('IC50 (log10 nM)'); plt.xlabel('log10(IC50_nM)'); plt.ylabel('Count'); plt.show()
plt.figure(); plt.hist(df['pIC50'], bins=50); plt.title('pIC50'); plt.xlabel('pIC50'); plt.ylabel('Count'); plt.show()

## Light SMILES validation

In [ ]:
def check_smiles_format(df_in, column_name='canonical_smiles'):
    df_local = df_in.copy()
    def bad(s):
        if not isinstance(s, str) or s.strip()=='' : return True
        if s.count('[')!=s.count(']'): return True
        if s.count('(')!=s.count(')'): return True
        if s.startswith('.') or s.endswith('.') or '.' in s: return True
        return False
    df_local['is_poorly_formatted'] = df_local[column_name].apply(bad)
    return df_local[df_local['is_poorly_formatted']]
poor = check_smiles_format(df, 'canonical_smiles'); print('Poor SMILES:', len(poor)); poor.head()

## RDKit descriptors

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
def compute_descriptors(smiles_series):
    rows, idxs, failed = [], [], []
    for idx, smi in smiles_series.items():
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            failed.append(idx); continue
        rows.append({
            'MolWt': Descriptors.MolWt(mol),
            'LogP': Descriptors.MolLogP(mol),
            'HBD': Descriptors.NumHDonors(mol),
            'HBA': Descriptors.NumHAcceptors(mol),
            'TPSA': Descriptors.TPSA(mol),
            'RotB': Descriptors.NumRotatableBonds(mol),
            'RingCount': Descriptors.RingCount(mol),
            'FractionCSP3': Descriptors.FractionCSP3(mol),
            'HeavyAtomCount': Descriptors.HeavyAtomCount(mol)
        }); idxs.append(idx)
    return pd.DataFrame(rows, index=idxs), failed
desc_df, failed = compute_descriptors(df['canonical_smiles'])
df_desc = df.loc[desc_df.index].reset_index(drop=True)
desc_df = desc_df.reset_index(drop=True)
data = pd.concat([df_desc, desc_df], axis=1)
print('Descriptor rows:', len(desc_df), '| Failed SMILES:', len(failed))
data.to_csv('EGFR_EDA_with_descriptors.csv', index=False)
data.head()

## Correlations and simple relationships

In [ ]:
descriptor_cols = ['MolWt','LogP','HBD','HBA','TPSA','RotB','RingCount','FractionCSP3','HeavyAtomCount']
corrs = data[descriptor_cols + ['pIC50']].corr()
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6)); im = plt.imshow(corrs, interpolation='nearest'); plt.title('Correlation matrix');
plt.xticks(range(len(corrs.columns)), corrs.columns, rotation=45, ha='right'); plt.yticks(range(len(corrs.index)), corrs.index);
plt.colorbar(im, fraction=0.046, pad=0.04); plt.tight_layout(); plt.show()
target_corr = corrs['pIC50'].drop('pIC50').sort_values(ascending=False); print(target_corr)
plt.figure(); plt.bar(target_corr.index, target_corr.values); plt.title('Descriptor correlation with pIC50');
plt.xticks(rotation=45, ha='right'); plt.ylabel('Pearson r'); plt.tight_layout(); plt.show()

## PCA & UMAP (structure in descriptor space)

In [ ]:
feat = data[descriptor_cols].replace([np.inf, -np.inf], np.nan).dropna()
scaler = StandardScaler(); Xz = scaler.fit_transform(feat.values)
pca = PCA(n_components=2, random_state=42); pca_xy = pca.fit_transform(Xz)
plt.figure(); plt.scatter(pca_xy[:,0], pca_xy[:,1], alpha=0.7); plt.xlabel('PC1'); plt.ylabel('PC2'); plt.title('PCA'); plt.show()
um = UMAP(n_components=2, random_state=42); um_xy = um.fit_transform(Xz)
plt.figure(); plt.scatter(um_xy[:,0], um_xy[:,1], alpha=0.7); plt.xlabel('UMAP-1'); plt.ylabel('UMAP-2'); plt.title('UMAP'); plt.show()

## Isolation Forest outliers (optional)

In [ ]:
iso = IsolationForest(contamination=0.05, random_state=42); labels = iso.fit_predict(Xz)
outliers = (labels == -1); print('Outliers flagged:', outliers.sum(), '/', len(outliers))
plt.figure();
plt.scatter(pca_xy[~outliers,0], pca_xy[~outliers,1], alpha=0.5)
plt.scatter(pca_xy[outliers,0], pca_xy[outliers,1], alpha=0.9, marker='x')
plt.xlabel('PC1'); plt.ylabel('PC2'); plt.title('Outliers on PCA'); plt.show()
data_out = data.loc[feat.index].copy(); data_out['pca_x']=pca_xy[:,0]; data_out['pca_y']=pca_xy[:,1];
data_out['umap_x']=um_xy[:,0]; data_out['umap_y']=um_xy[:,1]; data_out['is_outlier_iforest']=outliers;
data_out.to_csv('EGFR_EDA_with_embeddings.csv', index=False); print('Saved EGFR_EDA_with_embeddings.csv')